# Porting LSST test queries to Stripe82++ : part 2
Ani will do this one.

Notebook with queries representing LSST test queries, but against the Stripe82 database. The latter was extended with a 'match' and 'coadd' table following script in 
https://docs.google.com/document/d/1qFOE1s4bQUDM8W7wLzoxb2OK1LaPwzd5Lmy_GKUYut0.

Queries from https://dev.lsstcorp.org/trac/wiki/db/queries, second batch:
<pre>
Cone-magnitude-color search 
Select all galaxies in given area 
Select all variable objects in given area 
Select time series data for all objects in a given area of the sky, in a given photometric band with a given variability index 
How many objects cross both Jupiter and Saturn? 
Select all unresolved objects (stellar PSF) in a given region of sky, and return their attributes 
Select all unresolved object in a specified region with cuts on attributes 
For a specified patch of sky, give me the source count density of unresolved sources (star like morphology) 
</pre>

Assuming LSST schema S12_sdss from https://lsst-web.ncsa.illinois.edu/schema/index.php?sVer=S12_sdss

Ths notebook is in http://github/com/idies/jhu-lsst/Stripe82/test_queries/LSST-Test-Queries%20-%20Part%202.ipynb

In [6]:
import pandas
import matplotlib.pyplot as plt
import numpy as np
import skimage.io                   # image processing library


import SciServer.CasJobs as cj
import SciServer.SkyServer as skyS

In [7]:
# define some variable to be used in query execution
TARGET_CONTEXT="Stripe82_016"
raMin = "308"
raMax = "310"
decMin = "-1.5"
decMax = "-0.5"
zMin = "19"
zMax ="22"
grMin = "1"
grMax = "1.5"
izMin = "0.5"
izMax = "1"

### <a href="https://dev.lsstcorp.org/trac/wiki/db/queries/003" target="_blank">Cone-magnitude-color search</a>

https://dev.lsstcorp.org/trac/wiki/db/queries/003

QSERV
```sql
SELECT  *
FROM    Object
WHERE   areaSpec_box(:raMin, :declMin, :raMax, :declMax)
   AND  zMag      BETWEEN :zMin  AND :zMax
   AND  gMag-rMag BETWEEN :grMin AND :grMax
   AND  iMag-zMag BETWEEN :izMin AND :izMax
```

In [10]:
# Replaced the "select *" with a list to avoid a huge output.  Also added a "top 10". The HTM version is not working ATM,
# looks like the assembly is not initialized properly in the DB.
sql="""
SELECT  TOP 10 b.objID, b.ra, b.dec, b.u, b.g, b.r, b.i, b.z
FROM 
   -- dbo.fGetObjFromRectEq( 180, -0.5, 180.1, -0.4) a      -- HTM version
   -- JOIN PhotoObjAll b ON a.objID=b.objID                 -- HTM Version
   PhotoObjAll b 
WHERE
   ra between """+raMin+""" and """+raMax+"""
   AND dec between """+decMin+""" and """+decMax+"""
   AND  z BETWEEN """+zMin+""" and """+zMax+"""
   AND  g-r BETWEEN """+grMin+""" and """+grMax+"""
   AND  i-z BETWEEN """+izMin+""" and """+izMax+"""
"""
df=cj.executeQuery(sql,TARGET_CONTEXT)
#g=pandas.read_csv(gals)
df

,objID,ra,dec,u,g,r,i,z
0,8658201040424077601,308.000001,-1.203886,24.96831,23.77797,22.77130,21.42617,20.85246
1,8658197844966312106,308.000020,-1.227724,23.54187,24.15288,22.85626,21.51331,20.76353
2,8658197879863249413,308.000035,-0.667164,22.76138,24.70139,23.38732,22.03683,21.51190
3,8658198532162323371,308.000058,-0.842994,26.72729,22.89887,21.67011,20.76495,20.23203
4,8658198695371146091,308.000069,-1.219391,23.49597,21.46917,20.24049,19.63561,19.06381
5,8658197570088798700,308.000084,-1.227775,23.75058,24.35763,23.21495,21.72814,21.01948
6,8658476657062119692,308.000091,-1.264932,25.53100,24.68461,23.42423,21.51370,20.71667
7,8658200645818778634,308.000095,-0.624704,-9999.00000,21.52625,20.41356,20.02446,19.41243
8,8658198133266908533,308.000104,-0.624707,22.98230,21.58433,20.41374,20.01545,19.49387
9,8658197643641489173,308.000108,-0.624734,25.76686,21.37444,20.31774,19.97374,19.45280


In [ ]:
### <a href="https://dev.lsstcorp.org/trac/wiki/db/queries/014" target="_blank">Select all galaxies in given area</a>

https://dev.lsstcorp.org/trac/wiki/db/queries/014
    
QSERV
```sql
SELECT objectId
FROM   Object
WHERE  areaSpec_box(:raMin, :declMin, :raMax, :declMax)
AND    extendedParameter > 0.8
```

In [16]:
# Replaced "extendedParameter" with Petrosian radius in r band and ran it on Galaxy view.  
# Also added a "top 10". The HTM version is not working ATM, looks like the assembly is not initialized properly in the DB.
rMin = "2.5"
sql="""
SELECT  TOP 10 b.objID, b.ra, b.dec, b.petroRad_r
FROM 
   -- dbo.fGetObjFromRectEq( 180, -0.5, 180.1, -0.4) a      -- HTM version
   -- JOIN PhotoObjAll b ON a.objID=b.objID                 -- HTM Version
   Galaxy b 
WHERE
   ra between """+raMin+""" and """+raMax+"""
   AND dec between """+decMin+""" and """+decMax+"""
   AND petroRad_r > """+rMin+"""
"""
df=cj.executeQuery(sql,TARGET_CONTEXT)
#g=pandas.read_csv(gals)
df

,objID,ra,dec,petroRad_r
0,8658194323094438011,308.000001,-1.080312,2.969953
1,8658200920164206050,308.000001,-1.219354,2.729609
2,8658476657062119693,308.000002,-1.227750,4.583994
3,8658174750926046477,308.000002,-1.219350,4.738014
4,8658198588533179532,308.000003,-0.733367,2.970514
5,8658197844966310164,308.000003,-1.057900,2.969805
6,8658188335907932731,308.000004,-1.227764,2.970030
7,8658197973817952061,308.000006,-1.203791,2.969872
8,8658197879326378193,308.000010,-1.203811,2.969768
9,8658198064010494056,308.000013,-1.086952,16.800340


### <a href="https://dev.lsstcorp.org/trac/wiki/db/queries/008" target="_blank">Select all variable objects in given area</a>

https://dev.lsstcorp.org/trac/wiki/db/queries/008

QSERV
```sql
SELECT objectId
FROM   Object
WHERE  areaSpec(:raMin, :declMin, :raMax, :declMax)
AND    variability > 0.8
```

In [17]:
# Replaced "extendedParameter" with Petrosian radius in r band and ran it on Galaxy view.  
# Also added a "top 10". The HTM version is not working ATM, looks like the assembly is not initialized properly in the DB.
sql="""
SELECT  TOP 10 p.objID
FROM 
   -- dbo.fGetObjFromRectEq( 180, -0.5, 180.1, -0.4) o      -- HTM version
   -- JOIN PhotoTag p ON o.objID=p.objID                    -- HTM Version
   PhotoTag p 
WHERE
   p.ra between """+raMin+""" and """+raMax+"""
   AND p.dec between """+decMin+""" and """+decMax+"""
# need some kind of variability test
#  AND variabiliy > 0.8
"""
df=cj.executeQuery(sql,TARGET_CONTEXT)
#g=pandas.read_csv(gals)
df

Exception: Error when executing query. Http Response from CasJobs API returned status code 401:
{"Error Code":401,"Error Type":"Unauthorized","Error Message":"Authentication failed: Response status code does not indicate success: 500 (Internal Server Error).","LogMessageID":"cbbb293b-a553-4325-be31-0d27fed91690"}